In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import layers, Model
from tensorflow import keras

import os
import cv2
import glob
import matplotlib.pyplot as plt


In [ ]:
!pip install kaggle --upgrade
!pip install pandas numpy rasterio glob2

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -1ha kaggle.json

In [ ]:
!kaggle datasets download -d fpeccia/weed-detection-in-soybean-crops

In [ ]:
!unzip -q /content/weed-detection-in-soybean-crops.zip

In [ ]:
labels = ["broadleaf", "grass", "soil", "soybean"]

In [ ]:
X = []
Y = []
imgSize = (227, 227)
max_samples = 300
for i in labels:
  dataPath = "/content/dataset/" + i + "/"
  fileRead = glob.glob(dataPath + "*")[:max_samples]
  # print(fileRead)
  print(len(fileRead))

  for j in fileRead:
    image = cv2.imread(j)
    img = cv2.resize(image, imgSize)
    X.append(img)
    Y.append(i)



In [ ]:
X = np.array(X)
Y = np.array(Y)
  

In [ ]:
X.shape, Y.shape

In [ ]:
plt.imshow(X[0])

In [ ]:
oneHotY = []
for i in Y:
  oneHotY.append(labels.index(i))
Y = to_categorical(oneHotY)

In [ ]:
print(Y[0])
print(Y[1])
print(Y[2])
print(Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
#Model (AlexNet, Functional API)

inputs = layers.Input(shape = (227, 227, 3))

x = layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    # Convolutional Layer 2
x = layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    # Convolutional Layer 3
x = layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same")(x)
x = layers.BatchNormalization()(x)

    # Convolutional Layer 4
x = layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same")(x)
x = layers.BatchNormalization()(x)

    # Convolutional Layer 5
x = layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    # Flatten and dense layers
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs= outputs)
model.summary()


In [ ]:
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.01, momentum=0.9, decay=0.0005, nesterov=False)
loss_fn = 'categorical_crossentropy'
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=4, min_lr=0.0001)

In [ ]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
# train_history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=32)
import math
num_epochs = 100
batch_size = 32

train_steps_per_epoch = X_train.shape[0]// batch_size
val_steps_per_epoch = X_test.shape[0]// batch_size

hist = model.fit(X_train, Y_train,
                 epochs=num_epochs,
                 batch_size=batch_size,
                 verbose=1,
                 callbacks=[reduce_lr],
                 validation_data=(X_test, Y_test))

In [ ]:
acc = hist.history["accuracy"]
val_acc = hist.history["val_accuracy"]

loss = hist.history["loss"]
val_loss = hist.history["val_loss"]

acc = np.array(acc).T
epochs = [i for i in range(1, 101)]
plt.plot(epochs, acc, "-r.", label="Train acc")
plt.plot(epochs, val_acc, "-g", label="Val acc")
plt.title("Train & val acc.")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "-r.", label="Train loss")
plt.plot(epochs, val_loss, "-g", label="Val loss")
plt.title("Train & val loss")
plt.legend()
plt.show()